<a href="https://colab.research.google.com/github/hissain/ml/blob/main/codes/fine_tuning/Fine_Tuning_with_SFTTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture log
!pip install datasets
!pip install trl

In [2]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [3]:
from datasets import load_dataset
dataset = load_dataset("stanfordnlp/imdb", split="train")
subset = dataset.select(range(1000))
print(len(dataset))
print(len(subset))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

25000
1000


In [5]:

from trl import SFTConfig, SFTTrainer
training_args = SFTConfig(
    run_name='ft_test_1',
    max_seq_length=512,
    output_dir="/tmp",
    logging_dir="/tmp/logs",
    logging_steps=100,
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=subset,
    args=training_args,
)
trainer.train()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:202: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
100,3.430600
200,3.135100
300,2.956200


TrainOutput(global_step=375, training_loss=3.111963907877604, metrics={'train_runtime': 936.6033, 'train_samples_per_second': 3.203, 'train_steps_per_second': 0.4, 'total_flos': 2586255164571648.0, 'train_loss': 3.111963907877604, 'epoch': 3.0})

In [6]:
print("Training Loss:", trainer.state.log_history)

Training Loss: [{'loss': 3.4306, 'grad_norm': 5.550849437713623, 'learning_rate': 1.4666666666666666e-05, 'epoch': 0.8, 'step': 100}, {'loss': 3.1351, 'grad_norm': 6.074952125549316, 'learning_rate': 9.333333333333334e-06, 'epoch': 1.6, 'step': 200}, {'loss': 2.9562, 'grad_norm': 4.773190975189209, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.4, 'step': 300}, {'train_runtime': 936.6033, 'train_samples_per_second': 3.203, 'train_steps_per_second': 0.4, 'total_flos': 2586255164571648.0, 'train_loss': 3.111963907877604, 'epoch': 3.0, 'step': 375}]


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Ensure the input tensor and model are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer.model.to(device)  # Move model to the appropriate device

# Prepare input
input_text = "The movie was absolutely"
input_ids = trainer.tokenizer.encode(input_text, return_tensors="pt").to(device)  # Move input to same device

# Generate response
output = trainer.model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = trainer.tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Text:", generated_text)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Generated Text: The movie was absolutely terrible. I was expecting a good movie, but it was just awful. I was expecting a good movie, but it was just awful. I was expecting a good movie, but it was just awful. I was expecting a
